# Validate a CEX File

## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [1]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [2]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              

## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.


In [3]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source



import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}


import java.io.File

import java.io.PrintWriter


import scala.io.Source



### Useful Functions

Easily write a String to file:

In [4]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

defined function saveString

Given a path to a CEX file, return an `Option[Corpus]`. This lets us drill right down through a potentially complex CITE Library and get just the texts.

In [5]:
def corpusFromCex(cexPath: String): Option[Corpus] = {
    val lib: CiteLibrary = CiteLibrarySource.fromFile(cexPath)
    lib.textRepository match {
        case Some(tr) => Some(tr.corpus)
        case None => None
    }
}

defined function corpusFromCex

A useful function for printing out collections, vectors, etc.

In [6]:
def showMe(v:Any):Unit = {
  v match {
    case _:StringHistogram => {
        val toStringVec: Vector[String] = v.asInstanceOf[StringHistogram].histogram.map( h => {
            h.toString
        })
        println(s"""\n----\n${toStringVec.mkString("\n")}\n----\n""")
    }
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

defined function showMe

Given a Corpus and a URN, remove return a copy of the Corpus *missing* the identified passage.

In [7]:
def removePassage(c: Corpus, u: CtsUrn): Corpus = {
    val corpusToRemove = c ~~ u
    c -- corpusToRemove
}

defined function removePassage

## Load Corpora

We want to work with two versions of the *Encheiridion*: 

- The Greek text
- A lemmatized version of the Greek text

So let's load them now. We will make **three** `Corpus` objects. One for each of the versions, and one with both versions in the same Corpus

In [8]:
val epicGrkFilePath = "epictetus_encheiridion_greek_edition.cex"
val epicLemFilePath = "epictetus_encheiridion_greek_lemmatized.cex"

// Now we turn these into Corpus-objects.
// We will assume that the CEX files are present and correct… if anything goes wrong, we'll get an exception

// corpusFromCex returns an Option[Corpus], so we take a chance and `.get` the results

val epicGrk: Corpus = {
    corpusFromCex(epicGrkFilePath).get 
}

val epicLem: Corpus = {
    corpusFromCex(epicLemFilePath).get 
}


val epicAll: Corpus = {
    epicGrk ++ epicLem
}

/* But let's throw a little test in here */

val grkSize: Int = epicGrk.size
val lemSize: Int = epicLem.size
val allSize: Int = epicAll.size

assert( grkSize == lemSize )
assert ( (grkSize + lemSize) == allSize )


Nov 24, 2020 12:16:34 PM wvlet.log.Logger log
INFO: Building text repo from cex ...
Nov 24, 2020 12:16:34 PM wvlet.log.Logger log
INFO: Building collection repo from cex ...
Nov 24, 2020 12:16:34 PM wvlet.log.Logger log
INFO: Building relations from cex ...
Nov 24, 2020 12:16:34 PM wvlet.log.Logger log
INFO: All library components built.
Nov 24, 2020 12:16:34 PM wvlet.log.Logger log
INFO: Building text repo from cex ...
Nov 24, 2020 12:16:34 PM wvlet.log.Logger log
INFO: Building collection repo from cex ...
Nov 24, 2020 12:16:34 PM wvlet.log.Logger log
INFO: Building relations from cex ...
Nov 24, 2020 12:16:34 PM wvlet.log.Logger log
INFO: All library components built.


epicGrkFilePath: String = "epictetus_encheiridion_greek_edition.cex"
epicLemFilePath: String = "epictetus_encheiridion_greek_lemmatized.cex"
epicGrk: Corpus = Corpus(
  Vector(
    CitableNode(
      CtsUrn("urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:1.1"),
      "\u03c4\u1ff6\u03bd \u1f44\u03bd\u03c4\u03c9\u03bd \u03c4\u1f70 \u03bc\u03ad\u03bd \u1f10\u03c3\u03c4\u03b9\u03bd \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd, \u03c4\u1f70 \u03b4\u1f72 \u03bf\u1f50\u03ba \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd. \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd \u03bc\u1f72\u03bd \u1f51\u03c0\u03cc\u03bb\u03b7\u03c8\u03b9\u03c2, \u1f41\u03c1\u03bc\u03ae, \u1f44\u03c1\u03b5\u03be\u03b9\u03c2, \u1f14\u03ba\u03ba\u03bb\u03b9\u03c3\u03b9\u03c2 \u03ba\u03b1\u1f76 \u1f11\u03bd\u1f76 \u03bb\u03cc\u03b3\u1ff3 \u1f45\u03c3\u03b1 \u1f21\u03bc\u03ad\u03c4\u03b5\u03c1\u03b1 \u1f14\u03c1\u03b3\u03b1\u00b7 \u03bf\u1f50\u03ba \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd \u03b4\u1f72 \u03c4\u1f78 \u03c3\u1ff6\u

## How to find N-Grams

We can use the [OHCO2](https://cite-architecture.github.io/cite-api-docs/ohco2/api/edu/holycross/shot/ohco2/index.html) library to do n-gram queries on our Corpus-objects.

In the first example, we will look for N-Grams in the Greek text.

In [9]:
val n: Int = 3 // number of words in a pattern
val t: Int = 2 // occurring more than this many times in the Corpus

val ngh: StringHistogram = epicGrk.ngramHisto(n, t)

// Pretty-print the result
showMe(ngh)


----
11 οὐκ ἐφʼ ἡμῖν
7 τῶν οὐκ ἐφʼ
6 εἰ δὲ μή
3 τὰ παρὰ φύσιν
3 ἀλλὰ τὸ δόγμα
3 δόγμα τὸ περὶ
3 παρὰ φύσιν τῶν
3 κατὰ φύσιν ἔχουσαν
3 πιστὸν καὶ αἰδήμονα
3 ὅτι οὐ δεῖ
3 τὸ δόγμα τὸ
3 ἢ περὶ τὰ
3 ἐφʼ ἑκάστου τῶν
3 τῶν ἐφʼ ἡμῖν
----



n: Int = 3
t: Int = 2
ngh: StringHistogram = StringHistogram(
  Vector(
    StringCount(
      "\u03bf\u1f50\u03ba \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd",
      11
    ),
    StringCount("\u03c4\u1ff6\u03bd \u03bf\u1f50\u03ba \u1f10\u03c6\u02bc", 7),
    StringCount("\u03b5\u1f30 \u03b4\u1f72 \u03bc\u03ae", 6),
    StringCount(
      "\u03c4\u1f70 \u03c0\u03b1\u03c1\u1f70 \u03c6\u03cd\u03c3\u03b9\u03bd",
      3
    ),
    StringCount(
      "\u1f00\u03bb\u03bb\u1f70 \u03c4\u1f78 \u03b4\u03cc\u03b3\u03bc\u03b1",
      3
    ),
    StringCount(
      "\u03b4\u03cc\u03b3\u03bc\u03b1 \u03c4\u1f78 \u03c0\u03b5\u03c1\u1f76",
      3
    ),
    StringCount(
      "\u03c0\u03b1\u03c1\u1f70 \u03c6\u03cd\u03c3\u03b9\u03bd \u03c4\u1ff6\u03bd",
      3
    ),
    StringCount(
      "\u03ba\u03b1\u03c4\u1f70 \u03c6\u03cd\u03c3\u03b9\u03bd \u1f14\u03c7\u03bf\u03c5\u03c3\u03b1\u03bd",
      3
    ),
    StringCount(
      "\u03c0\u03b9\u03c3\u03c4\u1f78\u03bd \u03ba\u03b1\u1f76 \u03b1\u1f30\u0

The next example will show how to get a list of unique words, by frequency. We do this, simply, by using the same method but asking for "1-grams". We'll use a high threshold, for this example, so as not to make the list too long.

In [10]:
val n: Int = 1 // number of words in a pattern
val t: Int = 400 // occurring more than this many times in the Corpus

val ngh: StringHistogram = epicGrk.ngramHisto(n, t)

// Pretty-print the result
showMe(ngh)


----

----



n: Int = 1
t: Int = 400
ngh: StringHistogram = StringHistogram(Vector())

Let's compare vocabulary in the first half of the *Encheiridion* with vocabulary in the second half.

In [11]:
// We get two Corpuses, splitting the Greek text in half

val firstHalf: Corpus = {
    // We define the half with a citation
    val urn: CtsUrn = CtsUrn("urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:1-26")
    // We make a new Corpus by "twiddling" the Greek text
    epicGrk ~~ urn
}

val secondHalf: Corpus = {
    // We define the half with a citation
    val urn: CtsUrn = CtsUrn("urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:27-53")
    // We make a new Corpus by "twiddling" the Greek text
    epicGrk ~~ urn
}

// We get full vocabulary for both

val n: Int = 1 // number of words in a pattern
val t: Int = 0 // occurring more than this many times in the Corpus

val histo1: StringHistogram = firstHalf.ngramHisto(n, t)
val histo2: StringHistogram = secondHalf.ngramHisto(n, t)

// Let's map those so we just have the words. We don't need the counts right now.

val vocab1: Vector[String] = histo1.histogram.map(_.s)
val vocab2: Vector[String] = histo2.histogram.map(_.s)


firstHalf: Corpus = Corpus(
  Vector(
    CitableNode(
      CtsUrn("urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:1.1"),
      "\u03c4\u1ff6\u03bd \u1f44\u03bd\u03c4\u03c9\u03bd \u03c4\u1f70 \u03bc\u03ad\u03bd \u1f10\u03c3\u03c4\u03b9\u03bd \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd, \u03c4\u1f70 \u03b4\u1f72 \u03bf\u1f50\u03ba \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd. \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd \u03bc\u1f72\u03bd \u1f51\u03c0\u03cc\u03bb\u03b7\u03c8\u03b9\u03c2, \u1f41\u03c1\u03bc\u03ae, \u1f44\u03c1\u03b5\u03be\u03b9\u03c2, \u1f14\u03ba\u03ba\u03bb\u03b9\u03c3\u03b9\u03c2 \u03ba\u03b1\u1f76 \u1f11\u03bd\u1f76 \u03bb\u03cc\u03b3\u1ff3 \u1f45\u03c3\u03b1 \u1f21\u03bc\u03ad\u03c4\u03b5\u03c1\u03b1 \u1f14\u03c1\u03b3\u03b1\u00b7 \u03bf\u1f50\u03ba \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd \u03b4\u1f72 \u03c4\u1f78 \u03c3\u1ff6\u03bc\u03b1, \u1f21 \u03ba\u03c4\u1fc6\u03c3\u03b9\u03c2, \u03b4\u03cc\u03be\u03b1\u03b9, \u1f00\u03c1\u03c7\u03b1\u1f76 \u03ba\u03b1\u1f76 

Now that we have that data, we can ask questions of it.

In [12]:
// Comparing sizes

println(s"First Half: ${vocab1.size} unique words")
println(s"Second Half: ${vocab2.size} unique words")

// Comparing the 100 most frequent words

val mf1: Vector[String] = vocab1.take(100)
val mf2: Vector[String] = vocab2.take(100)

showMe(mf1.diff(mf2)) // will show words that mf1 has, but mf2 does not have

showMe(mf2.diff(mf1)) // will show words that mf2 has, but mf1 does not have


/* .diff can be confusing. Here is a very simple example. */

val t1 = Vector(1,2,3,4,5,6)

val t2 = Vector(4,5,6,7,8)

showMe(t1.diff(t2))

showMe(t2.diff(t1))


First Half: 974 unique words
Second Half: 1274 unique words

----
οὐχ
θέλεις
εὐθὺς
ὧν
ἵνα
ἔσῃ
ἄλλου
οὐδὲ
ἑκάστου
οἱ
δύνασαι
σὸν
ἴσθι
ἐπʼ
ἔχεις
πῶς
εὑρήσεις
γινόμενα
τινος
αἰδήμονα
σοὶ
ἔχει
μήτε
παῖδα
οὐδείς
πρόχειρον
διότι
ἔργον
παρὰ
ὃν
ἀλλότρια
πιστὸν
καλέσῃ
----


----
μηδὲ
πολὺ
ἀπὸ
σεαυτοῦ
νῦν
αὐτῷ
εἶτα
οἷον
αὐτὸς
κακῶς
γάρ
Σωκράτης
σῶμα
γε
ἐγώ
ἔργου
ἀγαθὸν
ἐκ
ποτε
καθάπερ
σεαυτῷ
ὑπὸ
ἑαυτοῦ
ἔξω
σεαυτὸν
ἄρα
αὐτὸν
εἴπῃς
πᾶσα
τινὸς
τὰς
κρείσσων
ἔτι
----


----
1
2
3
----


----
7
8
----



mf1: Vector[String] = Vector(
  "\u03ba\u03b1\u1f76",
  "\u03c4\u1f78",
  "\u03b4\u1f72",
  "\u03bc\u1f74",
  "\u1f22",
  "\u1f45\u03c4\u03b9",
  "\u03c4\u1f70",
  "\u1f02\u03bd",
  "\u03c4\u1ff6\u03bd",
  "\u1f10\u03c6\u02bc",
  "\u03bf\u1f56\u03bd",
  "\u1f10\u1f70\u03bd",
  "\u03b3\u1f70\u03c1",
  "\u1f10\u03bd",
  "\u03bf\u1f50",
  "\u1f21\u03bc\u1fd6\u03bd",
  "\u03b5\u1f30",
  "\u1f10\u03c0\u1f76",
  "\u03c4\u1f74\u03bd",
  "\u1f41",
  "\u03bf\u1f50\u03ba",
  "\u03c0\u03c1\u1f78\u03c2",
  "\u03c3\u03b5",
  "\u03c4\u03bf\u1fe6",
  "\u1f61\u03c2",
  "\u1f00\u03bb\u03bb\u02bc",
  "\u03b5\u1f36\u03bd\u03b1\u03b9",
  "\u1f00\u03bb\u03bb\u1f70",
  "\u03bf\u1f55\u03c4\u03c9",
  "\u03c4\u1ff7",
  "\u1f45\u03c4\u03b1\u03bd",
  "\u03bc\u1f72\u03bd",
  "\u03c4\u1f78\u03bd",
  "\u03c3\u03bf\u03b9",
  "\u1f10\u03c3\u03c4\u03b9\u03bd",
  "\u03c4\u03b9",
  "\u03bf\u1f50\u03b4\u1f72\u03bd",
  "\u03bc\u03ad\u03bc\u03bd\u03b7\u03c3\u03bf",
...
mf2: Vector[String] = Vector(
  "\u03ba\u03b1\u1f76",


## Most Different Passage?
    
What if we wanted to find passages that seem to be outliers? Perhaps we would look at only the 10 most common words. The algorithm…

- For each passage of the text…
- Get a Corpus of that passage, and a Corpus of the rest of the text, *minus* that passage.
- Find the 10 most common words in both Corpora.
- Do `.diff` in both directions; add the `.size` of the results.
- Sort the result. The passage with the highest score is the "most different" one.


In [13]:
// Define a "VocabDifference" object

case class VocabDifference(urn:CtsUrn, delta: Int)

// Generate a VocabDifference object

def diffVocab( urn: CtsUrn, c: Corpus ): VocabDifference = {
    val text: Corpus = removePassage( c, urn )
    val passage: Corpus = c ~~ urn
    
    val n: Int = 1 // number of words in a pattern
    val t: Int = 0 // occurring more than this many times in the Corpus
    val numToCompare: Int = 10 // How many words to compare

    val histoPassage: StringHistogram = passage.ngramHisto(n, t)
    //showMe(histoPassage.histogram.take(10))
    val histoText: StringHistogram = text.ngramHisto(n, t)
    //showMe(histoText.histogram.take(10))

    
    // Let's map those so we just have the words. We don't need the counts right now.

    val vocabText: Vector[String] = histoText.histogram.map(_.s).take(numToCompare)
    //showMe(vocabText)
    val vocabPassage: Vector[String] = histoPassage.histogram.map(_.s).take(numToCompare)
    //showMe(vocabPassage)


    val diff1: Int = vocabPassage.diff(vocabText).size
    val diff2: Int = vocabText.diff(vocabPassage).size
    
    val score: Int = diff1 + diff2
    
    //showMe(score)
    
    VocabDifference(urn, score)
}

val tu: CtsUrn = CtsUrn("urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:22")

diffVocab( tu, epicGrk)

defined class VocabDifference
defined function diffVocab
tu: CtsUrn = CtsUrn("urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:22")
res12_3: VocabDifference = VocabDifference(
  CtsUrn("urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:22"),
  18
)

So let's do this…

In [16]:
// List of passages to cycle through

val whichCorpus: Corpus = epicGrk

val allUrns: Vector[CtsUrn] = whichCorpus.urns

// But we want to go chapter-by-chapter, so we need to collapse the URNs

val useTheseUrns: Vector[CtsUrn] = allUrns.map(_.collapsePassageTo(1)).distinct

//showMe(useTheseUrns)

val diffList: Vector[VocabDifference] = {
    useTheseUrns.map( u => {
        diffVocab( u, whichCorpus)
    }).sortBy( _.delta).reverse
}

showMe(diffList)



----
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:1
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:2
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:3
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:4
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:5
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:6
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:7
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:8
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:9
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:10
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:11
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:12
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:13
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:14
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:15
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:16
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:17
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:18
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:19
urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:20
urn:cts:greekLit:tlg0557.tlg002.perseus-grc

whichCorpus: Corpus = Corpus(
  Vector(
    CitableNode(
      CtsUrn("urn:cts:greekLit:tlg0557.tlg002.perseus-grc1:1.1"),
      "\u03c4\u1ff6\u03bd \u1f44\u03bd\u03c4\u03c9\u03bd \u03c4\u1f70 \u03bc\u03ad\u03bd \u1f10\u03c3\u03c4\u03b9\u03bd \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd, \u03c4\u1f70 \u03b4\u1f72 \u03bf\u1f50\u03ba \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd. \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd \u03bc\u1f72\u03bd \u1f51\u03c0\u03cc\u03bb\u03b7\u03c8\u03b9\u03c2, \u1f41\u03c1\u03bc\u03ae, \u1f44\u03c1\u03b5\u03be\u03b9\u03c2, \u1f14\u03ba\u03ba\u03bb\u03b9\u03c3\u03b9\u03c2 \u03ba\u03b1\u1f76 \u1f11\u03bd\u1f76 \u03bb\u03cc\u03b3\u1ff3 \u1f45\u03c3\u03b1 \u1f21\u03bc\u03ad\u03c4\u03b5\u03c1\u03b1 \u1f14\u03c1\u03b3\u03b1\u00b7 \u03bf\u1f50\u03ba \u1f10\u03c6\u02bc \u1f21\u03bc\u1fd6\u03bd \u03b4\u1f72 \u03c4\u1f78 \u03c3\u1ff6\u03bc\u03b1, \u1f21 \u03ba\u03c4\u1fc6\u03c3\u03b9\u03c2, \u03b4\u03cc\u03be\u03b1\u03b9, \u1f00\u03c1\u03c7\u03b1\u1f76 \u03ba\u03b1\u1f7